In [1]:
# ===============================
# STEP 1: Mount Google Drive
# ===============================

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U torch transformers accelerate peft datasets bitsandbytes sentencepiece huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 20.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [ ]:
pip install openai-whisper pytesseract pillow ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=91767cf2953b99ace068effe5cf4246997a3e4f6c8c9191d1b6de1da6a03a18a
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=6758252a2af800ced0b48e98d5f04df4c5a37e70863fb5343c03497383d27b50
  Stored in directory: /root/.cache/pip/wheels/26/21/0c/c26e09dff860a9071683e279445262346e008a9a1d2142c4ad
Successfully built openai-whisper ffmpeg


In [ ]:
from huggingface_hub import login

import os
HF_TOKEN = os.getenv("HF_TOKEN")


# ==========================================
# STEP 4: Download Mistral model to Drive
# ==========================================
from huggingface_hub import snapshot_download

model_repo = "mistralai/Mistral-7B-Instruct-v0.2"
save_dir = "/content/drive/MyDrive/chainmind/model_loader_mistral"

snapshot_download(
    repo_id=model_repo,
    local_dir=save_dir,
    local_dir_use_symlinks=False,
    resume_download=True
)

print(f"✅ Model successfully downloaded to: {save_dir}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

✅ Model successfully downloaded to: /content/drive/MyDrive/chainmind/model_loader_mistral


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import os

# Path to your local model
local_model_path = "/content/drive/MyDrive/chainmind/model_loader_mistral"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Quantization config for GPU + CPU offload
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,                        # Use 8-bit quantization
    llm_int8_enable_fp32_cpu_offload=True     # Offload large layers to CPU when needed
)

# Folder for temporary offload
offload_folder = "/content/chainmind_offload"
os.makedirs(offload_folder, exist_ok=True)

# Load model on GPU with offloading enabled
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    quantization_config=bnb_config,
    device_map="auto",               # Automatically map layers to GPU/CPU
    offload_folder=offload_folder
)

print("✅ Mistral-7B-Instruct loaded successfully on GPU!")

# Generation example
prompt = "Explain why reinforcement learning is useful for autonomous vehicles."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Send tensors to GPU
outputs = model.generate(**inputs, max_new_tokens=150)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Mistral-7B-Instruct loaded successfully on GPU!
Explain why reinforcement learning is useful for autonomous vehicles.

Reinforcement learning (RL) is a type of machine learning algorithm that enables an agent to learn how to make decisions by interacting with its environment. In the context of autonomous vehicles, RL can be used to train the vehicle to make optimal driving decisions based on the current state of the environment and the desired goal.

Autonomous vehicles face complex decision-making scenarios, where they need to consider various factors such as traffic rules, pedestrian safety, road conditions, and other vehicles on the road. Traditional rule-based systems or hand-crafted features may not be able to capture the nuances and variability of real-world driving scenarios. RL, on the other hand, can learn from


In [ ]:
import json
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# -----------------------------
# 1. Load dataset
# -----------------------------
dataset_path = "/content/drive/MyDrive/chainmind/dataset/domain_classifier_dataset.json"
with open(dataset_path, "r") as f:
    data = json.load(f)

texts = [item['text'] for item in data]
labels = [item['domain'] for item in data]  # Use 'domain' key

# Map labels to integers
label_set = sorted(list(set(labels)))
label2id = {label: i for i, label in enumerate(label_set)}
id2label = {i: label for label, i in label2id.items()}
labels = [label2id[label] for label in labels]

# -----------------------------
# 2. Create PyTorch Dataset
# -----------------------------
class DomainDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# -----------------------------
# 3. Load DistilBERT tokenizer and model
# -----------------------------
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(label_set),
    id2label=id2label,
    label2id=label2id
)

# -----------------------------
# 4. Prepare datasets
# -----------------------------
dataset = DomainDataset(texts, labels, tokenizer)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# -----------------------------
# 5. Training arguments (old-compatible)
# -----------------------------
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/chainmind/DistilBERT",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='/content/drive/MyDrive/chainmind/DistilBERT/logs',
    logging_steps=1,  # Log every step
    save_total_limit=2,
    do_eval=True
)

# -----------------------------
# 6. Metrics function
# -----------------------------
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# -----------------------------
# 7. Custom callback for step-wise loss
# -----------------------------
class StepLossCallback(TrainerCallback):
    def __init__(self):
        self.epoch_loss = 0
        self.steps = 0

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and 'loss' in logs:
            self.epoch_loss += logs['loss']
            self.steps += 1
            print(f"Step {state.global_step}, Training Loss: {logs['loss']:.4f}")

    def on_epoch_end(self, args, state, control, **kwargs):
        if self.steps > 0:
            avg_loss = self.epoch_loss / self.steps
            print(f"Epoch {state.epoch} finished, Average Loss: {avg_loss:.4f}")
        self.epoch_loss = 0
        self.steps = 0

# -----------------------------
# 8. Trainer
# -----------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[StepLossCallback]
)

# -----------------------------
# 9. Train and save model
# -----------------------------
trainer.train()
trainer.save_model("/content/drive/MyDrive/chainmind/DistilBERT")
tokenizer.save_pretrained("/content/drive/MyDrive/chainmind/DistilBERT")

# -----------------------------
# 10. Testing / Inference
# -----------------------------
from transformers import pipeline
classifier = pipeline(
    "text-classification",
    model="/content/drive/MyDrive/chainmind/DistilBERT",
    tokenizer="/content/drive/MyDrive/chainmind/DistilBERT",
    top_k=None  # return all scores
)

# Test examples
test_texts = [
    "The doctor recommended a new treatment for diabetes.",
    "The court issued a ruling in favor of the plaintiff.",
    "We had a fun day at the park."
]

for text in test_texts:
    result = classifier(text)
    print(f"\nText: {text}")
    print(f"Prediction: {result}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1838390555.py:124: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,1.103000
2,1.081700
3,1.062400
4,1.023200
5,0.961900
6,0.947300
7,0.858400
8,0.864700
9,0.837400
10,0.713100


Step 1, Training Loss: 1.1030
Step 2, Training Loss: 1.0817
Step 3, Training Loss: 1.0624
Step 4, Training Loss: 1.0232
Step 5, Training Loss: 0.9619
Step 6, Training Loss: 0.9473
Step 7, Training Loss: 0.8584
Step 8, Training Loss: 0.8647
Step 9, Training Loss: 0.8374
Step 10, Training Loss: 0.7131
Epoch 1.0 finished, Average Loss: 0.9453
Step 11, Training Loss: 0.6578
Step 12, Training Loss: 0.5691
Step 13, Training Loss: 0.6528
Step 14, Training Loss: 0.5051
Step 15, Training Loss: 0.5374
Step 16, Training Loss: 0.4373
Step 17, Training Loss: 0.4831
Step 18, Training Loss: 0.4384
Step 19, Training Loss: 0.4720
Step 20, Training Loss: 0.3910
Epoch 2.0 finished, Average Loss: 0.5144
Step 21, Training Loss: 0.4896
Step 22, Training Loss: 0.3116
Step 23, Training Loss: 0.3391
Step 24, Training Loss: 0.3167
Step 25, Training Loss: 0.3061
Step 26, Training Loss: 0.2434
Step 27, Training Loss: 0.2520
Step 28, Training Loss: 0.2544
Step 29, Training Loss: 0.2675
Step 30, Training Loss: 0.22

Device set to use cpu



Text: The doctor recommended a new treatment for diabetes.
Prediction: [[{'label': 'medical', 'score': 0.8007672429084778}, {'label': 'general', 'score': 0.11073911935091019}, {'label': 'legal', 'score': 0.08849363029003143}]]

Text: The court issued a ruling in favor of the plaintiff.
Prediction: [[{'label': 'legal', 'score': 0.8628348112106323}, {'label': 'general', 'score': 0.07279344648122787}, {'label': 'medical', 'score': 0.06437170505523682}]]

Text: We had a fun day at the park.
Prediction: [[{'label': 'general', 'score': 0.7576426267623901}, {'label': 'medical', 'score': 0.13658010959625244}, {'label': 'legal', 'score': 0.10577719658613205}]]


In [ ]:
classifier = DomainClassifier("/content/drive/MyDrive/chainmind/DistilBERT")
print(classifier.classify("The doctor prescribed antibiotics after the surgery."))


In [ ]:
# ======================================
# LoRA Fine-Tuning for Mistral-7B-Instruct (Fixed)
# Dataset: chainmind_debate_dataset.json
# Output: /content/drive/MyDrive/chainmind/LoRA Adapter
# ======================================
import torch, json
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig,
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# ---------------------------
# 1. Paths and Model Loading
# ---------------------------
MODEL_PATH = "/content/drive/MyDrive/chainmind/model_loader_mistral"
DATA_PATH = "/content/drive/MyDrive/chainmind/dataset/chainmind_debate_dataset.json"
OUTPUT_PATH = "/content/drive/MyDrive/chainmind/LoRA Adapter"

# ---- Use new quantization config (no warnings) ----
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# ---- Load tokenizer and model ----
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16,
)

# ---------------------------
# 2. Prepare Dataset
# ---------------------------
def load_debate_data(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    samples = []
    for entry in data:
        topic = entry["topic"]
        for d in entry["debate"]:
            role = d["agent_role"]
            thought = d["thought"]
            response = d["response"]
            prompt = f"[ROLE: {role.upper()}]\nTopic: {topic}\nThought: {thought}\nResponse:"
            samples.append({"prompt": prompt, "response": response})
    return samples

dataset_list = load_debate_data(DATA_PATH)
dataset = Dataset.from_list(dataset_list)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

# ---- Fix the mapping issue (handles lists safely) ----
def tokenize_fn(example):
    prompt = example["prompt"]
    response = example["response"]

    # handle cases where prompt/response might be lists
    if isinstance(prompt, list):
        prompt = " ".join(prompt)
    if isinstance(response, list):
        response = " ".join(response)

    text = f"{prompt} {response}"
    tokens = tokenizer(
        text,
        truncation=True,
        max_length=1024,
        padding="max_length",
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_fn, batched=False, remove_columns=["prompt", "response"])

# ---------------------------
# 3. Apply LoRA Configuration
# ---------------------------
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ---------------------------
# 4. Training Configuration
# ---------------------------
from packaging import version
import transformers

transformers_version = transformers.__version__
print(f"Transformers version: {transformers_version}")

# ✅ Universal compatible TrainingArguments (works for 4.31 – 4.57+)
if version.parse(transformers_version) >= version.parse("4.55.0"):
    training_args = TrainingArguments(
        output_dir=OUTPUT_PATH,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=50,
        max_steps=500,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=2,
        report_to="none",
        do_eval=True,              # <-- new flag replaces evaluation_strategy
    )
else:
    training_args = TrainingArguments(
        output_dir=OUTPUT_PATH,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=50,
        max_steps=500,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        evaluation_strategy="steps",  # older syntax
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=2,
        report_to="none",
    )

# ---------------------------
# 5. Start Training
# ---------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

trainer.train()

# ---------------------------
# 6. Save LoRA Adapter
# ---------------------------
model.save_pretrained(OUTPUT_PATH)
tokenizer.save_pretrained(OUTPUT_PATH)

print(f"✅ LoRA adapter saved successfully to: {OUTPUT_PATH}")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

/tmp/ipython-input-3999213558.py:155: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940
Transformers version: 4.57.0


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,5.966200
20,5.899500
30,5.852600
40,5.799900
50,5.788900
60,5.761500
70,5.739000
80,5.727200
90,5.707000
100,5.700300


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

In [ ]:
import sys
import os

# Update this path to your project root
project_root = "/content/drive/MyDrive/chainmind"

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("✅ Project root added to sys.path:", project_root)

✅ Project root added to sys.path: /content/drive/MyDrive/chainmind


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
# Install required NLP and metrics packages
!pip install rouge-score textblob nltk transformers torch bert-score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5ab5e65ec7ce672f2e94103fca4abd6cb6677bea0afc739f25a97a9a50d1eb01
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
# test_combined_metrics.py

import sys
import importlib

# Ensure project root path
sys.path.append("/content/drive/MyDrive/chainmind")

# Import and reload to pick latest changes
import utils.combined_metrics as cm
importlib.reload(cm)

def test_combined_metrics():
    print("\n🚀 Running CombinedMetrics Evaluation Test...")

    # 🧠 Simulated debate transcript
    transcript = [
        {"agent": "Alex", "response": "AI can help therapists by analyzing behavioral patterns and providing early warning signs."},
        {"agent": "Casey", "response": "AI is not just analysis; it's an artist's brush painting human emotions into data."},
        {"agent": "Jordan", "response": "But relying on AI might distance people from real empathy, which no machine can replace."},
        {"agent": "Alex", "response": "Still, if used carefully, AI can make therapy more accessible and affordable."},
        {"agent": "Casey", "response": "It’s about synergy — the blend of human empathy and machine precision."},
        {"agent": "Jordan", "response": "Ethical oversight is key, ensuring AI tools do not dictate but assist human therapists."}
    ]

    # 🧾 Consensus summary
    consensus_summary = (
        "AI should support mental health professionals, not replace them, "
        "ensuring accessibility while preserving empathy and human connection."
    )

    # Initialize metrics evaluator
    metrics = cm.CombinedMetrics()

    # Run evaluation
    results = metrics.evaluate(transcript, consensus_summary)

    # ✅ Validate key metrics
    expected_metrics = [
        "Avg Sentence Length",
        "Lexical Diversity",
        "Perplexity",
        "BLEU",
        "ROUGE-1",
        "ROUGE-L",
        "BERTScore-F1"
    ]
    for metric in expected_metrics:
        assert metric in results, f"Missing metric: {metric}"

    print("\n✅ CombinedMetrics test completed successfully.")
    print("\n📊 Test Metrics Summary:")
    for k, v in results.items():
        print(f"{k}: {v}")


if __name__ == "__main__":
    test_combined_metrics()



🚀 Running CombinedMetrics Evaluation Test...


/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_


✅ CombinedMetrics test completed successfully.

📊 Test Metrics Summary:
BLEU: 1.4316805779882116e-155
ROUGE-1: 0.19575152245480387
ROUGE-L: 0.163087914742531
BERTScore-F1: 0.20771992206573486
Avg Sentence Length: 15.833333333333334
Response Lengths: {'min': 14, 'avg': np.float64(15.833333333333334), 'max': 18}
Lexical Diversity: 0.7263157894736842
Perplexity: None


In [ ]:
!touch input_handlers/__init__.py core/__init__.py utils/__init__.py

touch: cannot touch 'input_handlers/__init__.py': No such file or directory
touch: cannot touch 'core/__init__.py': No such file or directory
touch: cannot touch 'utils/__init__.py': No such file or directory


In [ ]:
# main.py

import sys
import os
import time

# --- Path Fix ---
project_root = os.path.abspath(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)
# -----------------

# --- Import project modules ---
from input_handlers.domain_classifier import DomainClassifier
from input_handlers.audio_input import AudioHandler
from input_handlers.image_input import handle_image_input
from core.agent import Agent
from core.iteration_logic import IterationManager
from utils.combined_metrics import CombinedMetrics  # ✅ Metrics integration
from core.consensus_agent import ConsensusAgent  # ✅ Consensus integration

# --- Color class for terminal display ---
class Colors:
    HEADER = "\033[95m"; BLUE = "\033[94m"; CYAN = "\033[96m"
    GREEN = "\033[92m"; YELLOW = "\033[93m"; RED = "\033[91m"
    END = "\033[0m"; BOLD = "\033[1m"; GREY = "\033[90m"

def ensure_nltk():
    """Download NLTK data once if missing (no-op if present)."""
    import nltk
    try:
        nltk.data.find("tokenizers/punkt")
    except LookupError:
        nltk.download("punkt")
    # Newer NLTK splits out punkt tables:
    try:
        nltk.data.find("tokenizers/punkt_tab")
    except LookupError:
        try:
            nltk.download("punkt_tab")
        except Exception:
            pass  # old NLTK versions won't have this; safe to ignore

def run_application():
    ensure_nltk()

    print("✅ All modules imported and ready.")
    print("Initializing core components...")
    classifier = DomainClassifier()
    audio_handler = AudioHandler()
    print("✅ Components initialized.")

    print(f"\n{Colors.BOLD}{Colors.HEADER}🧠 ChainMind: Multi-Agent Reasoning System{Colors.END}")
    print("=" * 40)

    # --- 1️⃣ Get user input ---
    print(f"{Colors.HEADER}\n📥 Choose input type:{Colors.END}")
    print("1. Text")
    print("2. Audio")
    print("3. Image")
    choice = input("Enter option (1/2/3): ").strip()

    input_text = None
    if choice == '1':
        input_text = input(f"{Colors.CYAN}Please enter your topic:\n> {Colors.END}").strip()
    elif choice == '2':
        input_text = audio_handler.upload_and_transcribe()
    elif choice == '3':
        input_text = handle_image_input()
    else:
        print(f"{Colors.RED}❌ Invalid choice.{Colors.END}")
        return

    if not input_text or not input_text.strip():
        print(f"{Colors.RED}⚠️ No valid input provided.{Colors.END}")
        return

    # --- 2️⃣ Classify domain ---
    domain, _, _ = classifier.classify(input_text)
    print(f"\n{Colors.CYAN}📊 Initializing agents for a '{domain.upper()}' debate...{Colors.END}")

    # --- 3️⃣ Initialize agents ---
    agents = [
        Agent(name="Alex", role="analytical"),
        Agent(name="Casey", role="creative"),
        Agent(name="Jordan", role="critical"),
    ]
    iteration_manager = IterationManager(agents=agents)

    print(f"{Colors.GREEN}\n🤖 Agents ready. Starting debate...{Colors.END}")
    time.sleep(0.5)

    # --- 4️⃣ Run debate ---
    final_history = iteration_manager.run_debate(initial_input=input_text, num_iterations=2)

    # --- 5️⃣ Display full transcript ---
    print(f"\n{Colors.BOLD}{Colors.BLUE}📝 Full Debate Transcript:{Colors.END}\n")
    for i, entry in enumerate(final_history):
        agent_name = entry['agent']
        role = entry.get('role', 'topic')
        response = entry['response']

        if agent_name == "Topic":
            color = Colors.YELLOW; emoji = "📌"; role_display = ""
        else:
            color = Colors.GREEN if role == "analytical" else Colors.CYAN if role == "creative" else Colors.RED
            emoji = "🧠" if role == "analytical" else "🎨" if role == "creative" else "⚖️"
            role_display = f" ({role.capitalize()})"

        print(f"{color}{i}. {emoji} {agent_name}{role_display}:{Colors.END} {response}\n")

    print("-" * 40)

    # --- 6️⃣ Generate conversation metrics (✅ transcript wired correctly) ---
    print(f"\n{Colors.BOLD}{Colors.HEADER}📈 Evaluating Combined Metrics...{Colors.END}")
    try:
        metrics = CombinedMetrics(transcript=final_history)
        metrics.evaluate()
        metrics.print_summary()
    except Exception as e:
        print(f"{Colors.RED} Combined metrics evaluation failed: {e}{Colors.END}")

    # --- 7️⃣ Generate consensus summary ---
    print(f"\n{Colors.BOLD}{Colors.HEADER}📌 Generating Consensus Summary...{Colors.END}")
    try:
        consensus_agent = ConsensusAgent()
        consensus_summary = consensus_agent.generate_consensus(final_history, max_new_tokens=200)
        print(f"\n{Colors.CYAN}Consensus Summary:{Colors.END}\n{consensus_summary}\n")
    except Exception as e:
        print(f"{Colors.RED}❌ Consensus generation failed: {e}{Colors.END}")

    print(f"{Colors.BOLD}{Colors.GREEN}✅ ChainMind session completed successfully!{Colors.END}\n")

if __name__ == "__main__":
    run_application()


✅ All modules imported and ready.
Initializing core components...
▶️ Loading Mistral-7B base model from /content/drive/MyDrive/chainmind/model_loader_mistral on cuda...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

▶️ Applying LoRA adapter from /content/drive/MyDrive/chainmind/LoRA Adapter/checkpoint-400...
✅ Model and LoRA adapter loaded successfully.
▶️ Initializing Whisper model... (This may download the model on first run)


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 58.0MiB/s]


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📊 Combined Metrics Summary:
- BLEU: 1.0
- ROUGE-1: 1.0
- ROUGE-L: 1.0
- BERTScore-F1: 0.9999997615814209
- Avg Sentence Length: 63.0
- Response Lengths: {'min': 40, 'avg': 63.0, 'max': 85}
- Lexical Diversity: 0.3439153439153439
- Perplexity: 22.29872401555379

📌 Generating Consensus Summary...

Consensus Summary:
The panel agrees that the use of antibiotics must be balanced carefully to save lives in the short term while preserving their effectiveness for future generations. They acknowledge the urgent need for new, less toxic classes of antibiotics and stress the importance of using them responsibly to avoid creating superbugs.

✅ ChainMind session completed successfully!



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
import os

directory_to_delete = "/content/drive/MyDrive/chainmind/model_loader_mistral"

if os.path.exists(directory_to_delete):
    print(f"Deleting directory: {directory_to_delete}")
    shutil.rmtree(directory_to_delete)
    print(f"✅ Directory deleted successfully: {directory_to_delete}")
else:
    print(f"Directory not found: {directory_to_delete}")

Deleting directory: /content/drive/MyDrive/chainmind/model_loader_mistral
✅ Directory deleted successfully: /content/drive/MyDrive/chainmind/model_loader_mistral


In [ ]:
import os
import shutil

for item in os.listdir("/content"):
    item_path = os.path.join("/content", item)
    try:
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)  # delete file or symlink
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)  # delete folder
        print(f"Deleted: {item_path}")
    except Exception as e:
        print(f"Error deleting {item_path}: {e}")

print("✅ All files in local disk cleared!")


Error deleting /content/drive: [Errno 125] Operation canceled: '/content/drive/.Encrypted/.shortcut-targets-by-id'
✅ All files in local disk cleared!
